# Best Practices

In [12]:
using HomotopyContinuation, LinearAlgebra
set_default_compile(:none)

:none

## Try to avoid determinants and minors 

For many problems it is necessary to formulate the condition that vectors are linearly dependent or that a matrix is singular.

**Examples**
* [Computing the narrowest bottleneck of a variety](https://www.juliahomotopycontinuation.org/examples/sampling_bottlenecks/)
* Sampling the discriminant of a parameterized system


Algebraic geometers in partcular, often reach to determinants and minors to formulate such a condition.
However, for homotopy continuation approaches this can result in numerically challenging systems.

**Why?**

Homotopy continuation is a *geometric* solution method. We have a solution to a system, slightly perturb the coefficients of our system and try to find again a solution to the perturbed system. As a general rule of thumb, the larger the degree of the system the more difficult it is to find the solution to the perturbed system.

**Better approach**

1) Singular matrices

Consider the case that we want to encode that an $n \times n$ matrix $J$ is singular. Instead of requiring $\det(J) = 0$ we use an additional n-dimensional vector $v$ and write the (partial) system
$$
\begin{bmatrix} J \\ J * v \\ \sum_i v_i - 1 \end{bmatrix}
$$

### Example:

In [13]:
@var x y a
F = System([x^3 + a*x*y + 1, y^4 + 3 * x - 2], parameters = [a])
J = jacobian(F)

2×2 Matrix{Expression}:
 a*y + 3*x^2    a*x
           3  4*y^3

In [14]:
# BAD approach
G_bad = System([expressions(F); det(J)])

System of length 3
 3 variables: a, x, y

 1 + a*x*y + x^3
 -2 + 3*x + y^4
 -3*a*x + 4*y^3*(a*y + 3*x^2)

In [15]:
# GOOD approach
@var v[1:2]
G_good = System([expressions(F); J * v; sum(v) - 1])

System of length 5
 5 variables: a, v₁, v₂, x, y

 1 + a*x*y + x^3
 -2 + 3*x + y^4
 v₁*(a*y + 3*x^2) + a*x*v₂
 3*v₁ + 4*y^3*v₂
 -1 + v₁ + v₂

2) Linear dependent vectors

If you want to model that two vectors $v$ and $w$ are linearly dependent is usually much better to just impose the condition that there exists an $\lambda \neq 0$ such that $v = \lambda w$.
If you use parameter homotopies and start with a start solution where $v$ and $w$ are dependent, then imposing the condition $\lambda \neq 0$ is often unnecessary.

## Parameter homotopies

A powerful feature of parameter homotopies is the possibility to use the offline-online approach.

**Recall:**

Our previous discussion gives us the following approach for repeatedly solving a parameterized polynomial system $F(x;p)$ with parameter space $\mathbb{C}^m$.

1. Compute all isolated solutions $S_q$ for a general $q \in \mathbb{C}^m$ by some method (offline part - needs to be done only once)
2. For each parameter value $p \in \mathbb{C}^m$ of interest
    1. Construct the homotopy $H(x, t) = F(x; t q + (1-t) p)$
    2. For each $s \in S_q$ track the solution from $t=1$ to $t=0$. The result is $S_p$.

Offline part:

In [16]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])

x₁, x₂ = randn(ComplexF64, 2)
q = [-(x₁ + x₂), x₁ * x₂]
S_q = [[x₁], [x₂]]

2-element Vector{Vector{ComplexF64}}:
 [0.2663202712744034 + 0.6864186388794614im]
 [0.7779845825624262 + 0.5872047571010558im]

In [17]:
write_parameters("quad_start_params.txt", q)
write_solutions("quad_start_sols.txt", S_q)

Online part:

In [18]:
q = read_parameters("quad_start_params.txt")

2-element Vector{ComplexF64}:
  -1.0443048538368296 - 1.2736233959805172im
 -0.19587522503752267 + 0.6904076484364798im

In [19]:
S_q = read_solutions("quad_start_sols.txt")

2-element Vector{Vector{ComplexF64}}:
 [0.2663202712744034 + 0.6864186388794614im]
 [0.7779845825624262 + 0.5872047571010558im]

In [20]:
p = [-2.5, 3]
R_p = solve(F, S_q; start_parameters = q, target_parameters = p)

Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (0 real)
• random_seed: 0x89af08c6


In [21]:
S_p = solutions(R_p)

2-element Vector{Vector{ComplexF64}}:
 [1.25 - 1.1989578808281798im]
 [1.25 + 1.19895788082818im]

<div style="border:2px gray; border-style:solid; padding: 0.5em"> This approach works best if the start parameters and target parameters are:
   <ul>
      <li>of the same scale, i.e., similar in magnitude</li>
      <li>from similar probability distributions</li>
  </ul>
</div>

<div style="border:2px gray; border-style:solid; padding: 0.5em"> Try to avoid "random" integers as e.g. produced by Macaulay2 since these result oft in numerically badly conditioned systems </div> 

## Sparse > dense or why elimination is not always good

If you are used to work with a symbolic computer algebra system, you are used to formulate your problem such that the problem has a minimal number of variables, e.g.,  by eliminating some of the variables.

The tradeoff there is often: less variables in exchange for a system with higher degrees and with more terms (i.e. "more dense").

For homotopy continuation methods this approach can actually be harmful since the resulting system can be subtanially more expensive to evaluate, numerically worse, and of higher degree than the original system.

#### Example:  Steiner's conic problem

Consider Steiner's conic problem of computing plane conics tangents to five given conics.

We can formulate the problem as

In [22]:
@var x[1:2]

f, u = dense_poly(x, 2; coeff_name = :u)

(u₆ + u₁*x₁^2 + u₃*x₂^2 + u₄*x₁ + u₅*x₂ + u₂*x₂*x₁, Variable[u₁, u₂, u₃, u₄, u₅, u₆])

In [23]:
# The coefficents of the five given conics
@var C[1:6, 1:5]
# The coordinates of the five points of tangency
@var P[1:2,1:5]

exprs = vcat(map(1:5) do i
    cᵢ = f(u => C[:,i])
    pᵢ = P[:,i]
    # The conic f and cᵢ are tangent at pᵢ
    # if f(pᵢ) = cᵢ(pᵢ) = det([∇f ∇Jᵢ])) = 0
    Jᵢ = differentiate([cᵢ, f], x)
    [
        f(x => pᵢ, u[6] => 1)
        cᵢ(x => pᵢ)
        det(Jᵢ)(x => pᵢ, u[6] => 1)
    ]
end...)

F = System(
        exprs;
        variables = [u[1:5]; vec(P)],
        parameters = vec(C)
)

System of length 15
 15 variables: u₁, u₂, u₃, u₄, u₅, P₁₋₁, P₂₋₁, P₁₋₂, P₂₋₂, P₁₋₃, P₂₋₃, P₁₋₄, P₂₋₄, P₁₋₅, P₂₋₅
 30 parameters: C₁₋₁, C₂₋₁, C₃₋₁, C₄₋₁, C₅₋₁, C₆₋₁, C₁₋₂, C₂₋₂, C₃₋₂, C₄₋₂, C₅₋₂, C₆₋₂, C₁₋₃, C₂₋₃, C₃₋₃, C₄₋₃, C₅₋₃, C₆₋₃, C₁₋₄, C₂₋₄, C₃₋₄, C₄₋₄, C₅₋₄, C₆₋₄, C₁₋₅, C₂₋₅, C₃₋₅, C₄₋₅, C₅₋₅, C₆₋₅

 1 + u₁*P₁₋₁^2 + u₃*P₂₋₁^2 + u₄*P₁₋₁ + u₅*P₂₋₁ + u₂*P₁₋₁*P₂₋₁
 C₆₋₁ + C₁₋₁*P₁₋₁^2 + C₃₋₁*P₂₋₁^2 + C₄₋₁*P₁₋₁ + C₅₋₁*P₂₋₁ + C₂₋₁*P₁₋₁*P₂₋₁
 (u₅ + u₂*P₁₋₁ + 2*u₃*P₂₋₁)*(C₄₋₁ + 2*C₁₋₁*P₁₋₁ + C₂₋₁*P₂₋₁) - (C₅₋₁ + C₂₋₁*P₁₋₁ + 2*C₃₋₁*P₂₋₁)*(u₄ + 2*u₁*P₁₋₁ + u₂*P₂₋₁)
 1 + u₁*P₁₋₂^2 + u₃*P₂₋₂^2 + u₄*P₁₋₂ + u₅*P₂₋₂ + u₂*P₁₋₂*P₂₋₂
 C₆₋₂ + C₁₋₂*P₁₋₂^2 + C₃₋₂*P₂₋₂^2 + C₄₋₂*P₁₋₂ + C₅₋₂*P₂₋₂ + C₂₋₂*P₁₋₂*P₂₋₂
 (u₅ + u₂*P₁₋₂ + 2*u₃*P₂₋₂)*(C₄₋₂ + 2*C₁₋₂*P₁₋₂ + C₂₋₂*P₂₋₂) - (C₅₋₂ + C₂₋₂*P₁₋₂ + 2*C₃₋₂*P₂₋₂)*(u₄ + 2*u₁*P₁₋₂ + u₂*P₂₋₂)
 1 + u₁*P₁₋₃^2 + u₃*P₂₋₃^2 + u₄*P₁₋₃ + u₅*P₂₋₃ + u₂*P₁₋₃*P₂₋₃
 C₆₋₃ + C₁₋₃*P₁₋₃^2 + C₃₋₃*P₂₋₃^2 + C₄₋₃*P₁₋₃ + C₅₋₃*P₂₋₃ + C₂₋₃*P₁₋₃*P₂₋₃
 (u₅ + u₂*P₁₋₃ + 2*u₃*P₂₋₃)*(C₄₋₃

In [24]:
mixed_volume(F)

27072

To solve the system symbolically, you would first eliminate the variables $P_{ij}$.

Then our system would reduce to a system $T$ of five equations in five unknowns of degree 6.

However, each polynomial in the system is dense and has 3210 terms (wrt to $u$) (This makes the system *very expensive* to evaluate).

**There are tradeoffs**

* System $T$ has only mixed volume $6^5=7776$ compared to mixed volume $27072$ for $F$.
* $F$ is substantially cheaper to evaluate and substantially faster for performing parameter homotopy

When you deal with parameterized systems the possibly larger mixed volume is often not a problem since you can use monodromy:

In [ ]:
monodromy_solve(F; target_solutions_count = 3264)


  tracked loops (queued):            397 (368)

  solutions in current (last) loop:  381 (3)

  generated loops (no change):       2 (0)
  generated loops (no change):       2 (0)

## Julia: Use local environments for your projects

Often you need to get back to your projects after a couple months or even years. It would be great if the program you wrote for your project would still work, wouldn't it? Or did you ever experience that things work on your computer but not on you collaborators?

Julia's package manager allows you create *local environments*. There you can list all you package dependencies (in a `Project.toml` file) and it automatically records the exact version used (in a `Manifest.toml`) file.

*Hands on demo*

## HC: Compiled and interpreted systems